In [1]:
import numpy as np
import tensorflow as tf, beamos
import Queue, sys

In [2]:
# Don't use all the VRAM!
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.15)
#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

# We're on CPU!
sess = tf.Session()

In [3]:
# Data variables
seq_length = 16
out_seq_length = 16
batch_size = 1
vocab_size = 26 + 1               # 0 for padding
embedding_dim = 26

# Network hyperparameters
memory_dim = 200
num_layers = 1

# Training variables
epochs = 10

First build input placeholders and constants. The `seq2seq` API generally deals with lists of tensors, where each tensor represents a single timestep. An input to an embedding encoder, for example, would be a list of `seq_length` tensors, each of which is of dimension `batch_size` (specifying the embedding indices to input at a particular timestep).

We allocate a `labels` placeholder using the same convention. A `weights` constant specifies cross-entropy weights for each label at each timestep.

In [4]:
enc_inp = [tf.placeholder(tf.int32, shape=(None,), name="inp%i" % t) for t in range(seq_length)]
labels = [tf.placeholder(tf.int32, shape=(None,), name="labels%i" % t) for t in range(out_seq_length)]
weights = [tf.ones_like(labels_t, dtype=tf.float32) for labels_t in labels]
use_initial = tf.placeholder(tf.int32, shape=[], name="use_initial")
supplied_prev  = tf.placeholder(tf.float32, shape=(1, 28), name="supplied_prev")
supplied_state = tf.placeholder(tf.float32, shape=(1, memory_dim*2), name="supplied_state")
supplied_attns = tf.placeholder(tf.float32, shape=(1, memory_dim), name="supplied_attns")
supplied_encoder_state = tf.placeholder(tf.float32, shape=(1, memory_dim*2), name="supplied_encoder_state")
supplied_encoder_outputs = [tf.placeholder(tf.float32, shape=(1, memory_dim), name="supplied_encoder_outputs%i" % t) for t in range(seq_length)]

# Decoder input: DO NOT PREPEND GO TOKEN! (Feed it in during sampling.)
dec_inp = [tf.placeholder(tf.int32, shape=(None,), name="dec_inp%i" % t) for t in range(out_seq_length)]

# Initial memory value for recurrence.
prev_mem = tf.zeros((batch_size, memory_dim))

Build the sequence-to-sequence graph.

There is a **lot** of complexity hidden in these two calls, and it's certainly worth digging into both in order to really understand how this is working.

In [5]:
constituent_cell = tf.nn.rnn_cell.BasicLSTMCell(memory_dim)

if num_layers > 1:
    cell = tf.nn.rnn_cell.MultiRNNCell([constituent_cell] * num_layers)
else:
    cell = constituent_cell

In [7]:
encoder_outputs, encoder_state = beamos.embedding_attention_encoder_seq2seq(enc_inp, cell, vocab_size+1, embedding_dim)
output, state, attns = beamos.embedding_attention_seq2seq_beam(dec_inp, use_initial, supplied_prev, supplied_state, supplied_attns, cell, vocab_size+1, embedding_dim, supplied_encoder_outputs, supplied_encoder_state)

In [8]:
saver = tf.train.Saver(tf.all_variables(), max_to_keep=5)

# Restore variables
Optionally restore variables

In [9]:
resume_at = 650

if resume_at > 0:
    saver.restore(sess, 'checkpoints/saved-model-1off-attn-' + str(resume_at))

# Train

Do not initialize variables if restoring from a saved file.  
**Warning:** epoch numbers start from 0, and *will* overwrite your old saves!

In [10]:
if resume_at == 0:
    sess.run(tf.initialize_all_variables())

In [11]:
# Load data
train_x = np.load('data/mutated-train.npy')
train_y = np.load('data/fixes-train.npy')

assert(len(train_x) == len(train_y))
num_train = len(train_x)
print num_train

valid_x = np.load('data/mutated-validation.npy')
valid_y = np.load('data/fixes-validation.npy')

assert(len(valid_x) == len(valid_y))
num_validation = len(valid_x)
print num_validation

2080
640


In [15]:
beam_size = 5

def validate_batch(batch_id):
    X = valid_x[batch_id*batch_size:(batch_id+1)*batch_size]
    Y = valid_y[batch_id*batch_size:(batch_id+1)*batch_size]
    
    # Dimshuffle to seq_len * batch_size
    X = np.array(X).T
    Y = np.array(Y).T
    
    Y_hat = np.zeros(out_seq_length)
    
    q = Queue.PriorityQueue()
    q.put((0, [], []), False)
            
    feed_dict = {enc_inp[t]: X[t] for t in range(seq_length)}
    
    enc_outs = [0] * seq_length

    lhs = ""
    rhs = ""

    for i in range(seq_length):
        lhs += "enc_outs[%d], " % i
        rhs += "encoder_outputs[%d], " % i

    statement = "%s, enc_state = sess.run((%s, encoder_state), feed_dict)" % (lhs[:-2], rhs[:-2])
    exec(statement) in globals(), locals()
    
    feed_dict = {supplied_encoder_outputs[t]: enc_outs[t] for t in range(seq_length)}
    feed_dict.update({supplied_encoder_state: enc_state})
    
    # Do it one symbol at a time
    for t in range(out_seq_length):
        q_next = Queue.PriorityQueue()
        
        # Discard entries
        while q.qsize() > beam_size:
            q.get()
        
        while not q.empty():
            l, s, _ = q.get()
            
            for t2 in range(len(s)+1):
                if t2 == 0:
                    feed_dict.update({use_initial: 1})
                    feed_dict.update({dec_inp[0]: [0]})
                    # Useless:
                    feed_dict.update({supplied_state: np.zeros((1, memory_dim * 2))})
                    feed_dict.update({supplied_prev: np.zeros((1, vocab_size + 1))})
                    feed_dict.update({supplied_attns: np.zeros((1, memory_dim))})
                else:
                    feed_dict.update({use_initial: 0})
                    feed_dict.update({supplied_state: prev_state})
                    feed_dict.update({supplied_prev: prev_output})
                    feed_dict.update({supplied_attns: prev_attns})
                    feed_dict.update({dec_inp[0]: [s[t2-1]]})

                prev_output, prev_state, prev_attns = sess.run((output, state, attns), feed_dict)
                Y_hat[t2] = [logits_t.argmax() for logits_t in prev_output][0]
            
            probs = (prev_output[0]-np.min(prev_output[0]))
            probs = probs/np.sum(probs)
            bests = np.argpartition(probs, -beam_size)[-beam_size:]
            
            for best in bests:
                q_next.put((l + np.log(probs[best]), s + [best], Y_hat))
                
        q = q_next
    
    # Yes, there are `beam_size` of them!
    Y_hats = []
    
    # Discard entries
    while q.qsize() > beam_size:
        q.get()
        
    while not q.empty():
        _, this_y_hat, _ = q.get()
        Y_hats.append(this_y_hat)
 
    return X, Y.T[0], Y_hats

# Full Sequence Accuracy

In [16]:
accurate = 0
total = 0

for i in [64, 412, 571]:
    X, Y, Y_hats = validate_batch(i)
    
    for Y_hat in Y_hats:
        if np.array_equal(Y, Y_hat):
            accurate += 1
            break
    
    total += batch_size
    print "%d/%d" % (accurate, total)

1/1
2/2
3/3


In [17]:
float(accurate)/total

1.0